In [1]:
import numpy as np 
import pandas as pd

In [2]:
movies=pd.read_csv("C:\\Users\\mahal\\Desktop\\ALL\\General Minal\\ML\\movierecommend\\tmdb_5000_movies.csv")
credits=pd.read_csv("C:\\Users\\mahal\\Desktop\\ALL\\General Minal\\ML\\movierecommend\\tmdb_5000_credits.csv")

In [3]:
movies = movies.merge(credits,on='title')

In [4]:
#genres
#id
#keywords
#overview
#title
#cast
#crew

In [5]:
movies=movies[['id','genres','keywords','overview','title','cast','crew']];

In [6]:
movies.isnull().sum()

id          0
genres      0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [7]:
movies.dropna(inplace=True)

In [8]:
movies.duplicated().sum()

0

In [9]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [10]:
import ast

In [11]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [12]:
movies['genres']=movies['genres'].apply(convert)

In [13]:
movies['keywords']=movies['keywords'].apply(convert)

In [14]:
def convert3(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            l.append(i['name'])
        counter+=1;
        
    return l

In [15]:
movies['cast']=movies['cast'].apply(convert3)

In [16]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [17]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [18]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [19]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [20]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])

In [21]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [22]:
movies['tags']=movies['keywords']+ movies['overview']+movies['cast']+movies['crew']+movies['genres']

In [23]:
new_df=movies[['id','title','tags']]

In [24]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\mahal\AppData\Local\Temp\ipykernel_18820\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [25]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\mahal\AppData\Local\Temp\ipykernel_18820\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [26]:
import nltk

In [27]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [28]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [29]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\mahal\AppData\Local\Temp\ipykernel_18820\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000,stop_words='english')

In [31]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
similarity = cosine_similarity(vectors)

In [34]:
similarity

array([[1.        , 0.06819943, 0.07845987, ..., 0.04313311, 0.        ,
        0.        ],
       [0.06819943, 1.        , 0.0766965 , ..., 0.02108185, 0.        ,
        0.02222222],
       [0.07845987, 0.0766965 , 1.        , ..., 0.02425356, 0.        ,
        0.        ],
       ...,
       [0.04313311, 0.02108185, 0.02425356, ..., 1.        , 0.04264014,
        0.0421637 ],
       [0.        , 0.        , 0.        , ..., 0.04264014, 1.        ,
        0.08989331],
       [0.        , 0.02222222, 0.        , ..., 0.0421637 , 0.08989331,
        1.        ]])

In [35]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

744

In [36]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances=similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [37]:
recommend('Avatar')

Aliens vs Predator: Requiem
Predator
Battle: Los Angeles
Independence Day
Titan A.E.


In [38]:
import pickle

In [39]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [40]:
pickle.dump(similarity,open('similarity.pkl','wb'))